## 용어
- 변수 간 상관(correlated variables): 변수들이 같은 방향으로 움직이려는 경향을 가짐. 예측 변수들끼리 서로 높은 상관성을 가질 때는 개별 계수를 해석하는 것이 어렵다.
- 다중공선성(multicollinearity): 예측변수들이 완벽하거나 거의 완벽에 가까운 상관성을 갖는다고 할 때, 회귀는 불안정하며 계산이 불가능하다.
- 교란변수(confounding variable): 중요한 예측변수이지만 회귀방정식에 누락되어 결과를 잘못되게 이끄는 변수
- 주효과(main effect): 다른 변수들과 독립된, 하나의 예측변수와 응답변수 사이의 상호 의존적인 관계

In [6]:
import pandas as pd

from sklearn.linear_model import LinearRegression

from dmba import stepwise_selection, AIC_score

In [2]:
house = pd.read_csv('../../data/house_sales.csv', sep='\t')

## 예측변수 간 상관

In [10]:
features = ['SqFtTotLiving', 'SqFtLot', 'Bathrooms', 'Bedrooms',
              'BldgGrade', 'PropertyType', 'NbrLivingUnits',
              'SqFtFinBasement', 'YrBuilt', 'YrRenovated', 
              'NewConstruction']
label = 'AdjSalePrice'

X = pd.get_dummies(house[features], # 숫자형이 아닌 모든 컬럼들에 대해서 원핫인코딩 
                   drop_first=True) # 첫번째 카테고리는 사용하지 않음(나머지 값이 전부 0이면 첫번째 카테고리 값을 알수있다.)
X['NewConstruction'] = X['NewConstruction'].replace({False:0, True:1})
y = house[label]

# 주어진 변수 집합에 대해 적합 모델을 반환하는 함수를 정의
def train_model(variables):
    if len(variables) == 0:
        return None
    model = LinearRegression()
    model.fit(X[variables], y)
    return model

# 주어진 모델과 변수 세트에 대한 점수를 반환하는 함수를 정의
def score_model(model, variables):
    if len(variables) == 0:
        return AIC_score(y, [y.mean()]*len(y), model, df=1)
    return AIC_score(y, model.predict(X[variables]), model)

best_model, best_variables = stepwise_selection(X.columns, train_model, score_model, verbose=True)

print()
print(f'Intercept: {best_model.intercept_:.3f}')
print('Coefficients:')
for name, coef in zip(best_variables, best_model.coef_):
    print(f" {name}: {coef}")
    


#  * Bedrooms의 계수가 음수인 것을 볼 수 있다.
# => 침실 개수를 늘릴수록 그 가치가 감소한다.
# => 집이 클수록 침실이 더 많은 경향이 있으며, 침실 수보다는 주택의 크기가 주택 가격에 더 큰 영향을 준다.
#    똑같은 크기의 두 집이 있다고 하면, 작은 크기의 침실이 여러 개 있는 것을 선호하지 않는 것이 합리적이다.

Variables: SqFtTotLiving, SqFtLot, Bathrooms, Bedrooms, BldgGrade, NbrLivingUnits, SqFtFinBasement, YrBuilt, YrRenovated, NewConstruction, PropertyType_Single Family, PropertyType_Townhouse
Start: score=647988.32, constant
Step: score=633013.35, add SqFtTotLiving
Step: score=630793.74, add BldgGrade
Step: score=628230.29, add YrBuilt
Step: score=627784.16, add Bedrooms
Step: score=627602.21, add Bathrooms
Step: score=627525.65, add PropertyType_Townhouse
Step: score=627525.08, add SqFtFinBasement
Step: score=627524.98, add PropertyType_Single Family
Step: score=627524.98, unchanged None

Intercept: 6178645.017
Coefficients:
 SqFtTotLiving: 199.2775530420158
 BldgGrade: 137159.5602261976
 YrBuilt: -3565.4249392494557
 Bedrooms: -51947.383673614146
 Bathrooms: 42396.16452772052
 PropertyType_Townhouse: 84479.1620329995
 SqFtFinBasement: 7.046974967583083
 PropertyType_Single Family: 22912.05518701769


In [17]:
# Bedrooms과 상관관계가 있는 SqFtTotLiving, SqFtFinBasement, Bathrooms 제거후 계수

features = ['Bedrooms', 'BldgGrade', 'PropertyType', 'YrBuilt']
label = 'AdjSalePrice'

X = pd.get_dummies(house[features], drop_first=True)
y = house[label]

reduced_lm = LinearRegression()
reduced_lm.fit(X, y)

print(f'Intercept: {reduced_lm.intercept_:.3f}')
print('Coefficients:')
for name, coef in zip(X.columns, reduced_lm.coef_):
    print(f' {name}: {coef}')
    
# Bedrooms에 대한 계수가 양수인 것을 확인할 수 있음.

Intercept: 4913973.344
Coefficients:
 Bedrooms: 27150.537230219703
 BldgGrade: 248997.79366212635
 YrBuilt: -3211.7448621551157
 PropertyType_Single Family: -19898.495340501973
 PropertyType_Townhouse: -47355.436873344275


## 다중공선성